In [1]:
import spacy
import json
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
import pandas as pd

nlp = spacy.load('en_core_web_lg')
# from spacy.lang.en import English
# from spacy.pipeline import EntityRuler

In [2]:
def load_data(file):
    data = pd.read_csv(file, encoding='latin1', index_col = False)
    return(data)

def save_data(file,data):
    with open(file, "w",encoding="utf-8") as f:
        json.dump(data, f, indent = 4)

        
# format for training data > Train_data = [(text, {"entities":[(start,end,label)]})]

def test_model(text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities = entities + [(ent.start_char, ent.end_char, ent.label_)]
    if len(entities) > 0:
        results = [text,{"entities": entities}]
#         print(results)
    return(results)

def get_position(txt, str):
    start = txt.find(str)
    end = txt.find(str)+len(str)
    return start,end

data = load_data("tagged data.csv")

import numpy as np
data = data.replace(np.nan, '', regex=True)
data.head()



,NOTES,entity_age,category_age,entity_object,category_object
0,the Building is only 8 Years old and They are ...,8 years old,age,building,building
1,the Ia noted that the Contractor took one ol...,old,old,,
2,"insured is in Process of building a new Home, ...",new,new,home,building
3,"insured is in Process of building a new Home, ...",old,old,home,building
4,the referenced Roof Leaks were the Result of o...,old,old,roof,roof


In [1]:
txt = "the Building is only 8 Years old and They are reusing the Bricks old."
print(txt)
get_position(txt,'8 Years old')

TRAIN_DATA = []
data1 = data


for i in range(len(data1)):
    entities = []
    text = data1['NOTES'][i].strip()
    if data1['entity_age'][i]!= '':
        start_char, end_char = get_position(text.lower(),data1['entity_age'][i].lower())
        entities = [(start_char, end_char, data1['category_age'][i])]
    if data1['entity_object'][i] != '':
        start_char1, end_char1 = get_position(text.lower(),data1['entity_object'][i].lower())
        entities = entities + [(start_char1, end_char1, data1['category_object'][i])]
    results = [text,{"entities": entities}]
    TRAIN_DATA = TRAIN_DATA + [results]
    

the Building is only 8 Years old and They are reusing the Bricks old.


NameError: name 'get_position' is not defined

In [4]:
# TRAIN_DATA = []
# with open("hpchapter1.txt","r",encoding="utf-8") as f:
#     text = f.read()
#     chapters = text.split("CHAPTER")[1:]
#     for chapter in chapters:
#         chapter_num, chapter_title = chapter.split("\n\n")[0:2]
#         segments = chapter.split("\n\n")[2:]
#         results = list()
#         for segment in segments:
#             segment = segment.replace("\n"," ").strip()
#             result = test_model(nlp,segment)
#             if result != []:
#                 results = results + [result]
#         TRAIN_DATA = TRAIN_DATA + results
print(len(TRAIN_DATA))


129


In [5]:
TRAIN_DATA[0]

['the Building is only 8 Years old and They are reusing the Bricks .',
 {'entities': [(21, 32, 'age'), (4, 12, 'building')]}]

In [6]:
save_data("train_data.json",TRAIN_DATA)
for a , b in TRAIN_DATA:
    print(b)

TRAIN_DATA[4]

{'entities': [(21, 32, 'age'), (4, 12, 'building')]}
{'entities': [(42, 45, 'old')]}
{'entities': [(36, 39, 'new'), (40, 44, 'building')]}
{'entities': [(60, 63, 'old'), (40, 44, 'building')]}
{'entities': [(2085, 2088, 'old'), (15, 19, 'roof')]}
{'entities': [(9, 21, 'age')]}
{'entities': [(16, 23, 'new'), (5, 9, 'building')]}
{'entities': [(46, 58, 'age'), (28, 36, 'building')]}
{'entities': [(14, 17, 'old'), (0, 8, 'building')]}
{'entities': [(378, 395, 'age'), (68, 73, 'equipment')]}
{'entities': [(266, 269, 'old'), (270, 278, 'building')]}
{'entities': [(266, 269, 'old'), (283, 287, 'plumbing')]}
{'entities': [(11, 14, 'old'), (15, 28, 'building')]}
{'entities': [(202, 213, 'age'), (53, 58, 'equipment')]}
{'entities': [(287, 300, 'age'), (264, 272, 'building')]}
{'entities': [(841, 854, 'age'), (818, 826, 'building')]}
{'entities': [(287, 300, 'age'), (264, 272, 'building')]}
{'entities': [(-1, 3, 'age'), (18, 23, 'building')]}
{'entities': [(14, 17, 'old'), (18, 23, 'building')]}

['the referenced Roof Leaks were the Result of one or more of the Following:-improper Repair Methods and materials;-deteriorated Condition of the Roof covering as a Result of normal Wear and Weathering;-workmanship Deficiencies associated with the Installation of the Roof Assembly; and - improper Maintenance of the Roof assembly.*the referenced stained and/or deteriorated Portions of Ceiling finishes within the Dining Room, Kitchen, Guest Bathroom and southeast Bedroom were not the Result of Storm - created Openings in the Roof Assembly attributable to Forces associated with severe Weather Events ( e.g. Wind damage).*based on the observed Damage, the Roof Leaks above Dining Room, Kitchen, Guest Bathroom and southeast Bedroom were long - Term, repetitive and ongoing over a Period of 1 Year or more prior to the reported Date of loss.*the preceding Opinions were based on the Following:-ghd?s Observations; including but not limited to the Following*the Age and Condition of the Roof Coverin

In [7]:
### create a function that will be used to train the model 
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    
    for a , annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(other_pipes):
        optimizer = nlp.begin_training()
        
        for itn in range(iterations):
            print("starting iteration: " + str(itn))
            
            for itn in range(iterations):
                random.shuffle(TRAIN_DATA)
                losses = {}
                
                for batch in minibatch(TRAIN_DATA, size=8):
                    for text, annotations in batch:
                        doc = nlp.make_doc(text)
                        example = Example.from_dict(doc, annotations)
                        print('check1')
                        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
                    print(losses)
    return(nlp)

In [ ]:
nlp = train_spacy(TRAIN_DATA, 5)

starting iteration: 0
check1


In [ ]:
nlp.to_disk("age_model")